In [ ]:
import numpy as np

import tensorflow as tf
import time
import os
import glob
import PIL
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import cv2
import pandas as pd
import tensorflow.keras
from tensorflow.keras.utils import to_categorical
from keras.preprocessing.image import img_to_array,ImageDataGenerator
from tensorflow.keras.models import Sequential, Model
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
# TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
# TPU_ADDRESS

In [ ]:
#Run this cell after changing a runtime
#Run this cell if no files are seen in /content directory

import os
os.environ['KAGGLE_USERNAME'] = "obiwanshinobi" # username from the json file
os.environ['KAGGLE_KEY'] = "84788f016bcbae9fe939142cd6d8083a" # key from the json file

!kaggle competitions download -c galaxy-zoo-the-galaxy-challenge

!unzip -q \*.zip
!rm *.zip

In [ ]:
DIR = "/content/training_solutions_rev1.csv"
 
train_path = "/content/images_training_rev1"

In [ ]:
classes = [
    'Class1.1', 'Class1.2', 'Class1.3', 'Class2.1', 'Class2.2', 'Class3.1',
    'Class3.2', 'Class4.1', 'Class4.2', 'Class5.1', 'Class5.2', 'Class5.3',
    'Class5.4', 'Class6.1', 'Class6.2', 'Class7.1', 'Class7.2', 'Class7.3',
    'Class8.1', 'Class8.2', 'Class8.3', 'Class8.4', 'Class8.5', 'Class8.6',
    'Class8.7', 'Class9.1', 'Class9.2', 'Class9.3', 'Class10.1', 'Class10.2',
    'Class10.3', 'Class11.1', 'Class11.2', 'Class11.3', 'Class11.4',
    'Class11.5', 'Class11.6'
]


def append_ext(fn):
    return fn + ".jpg"


traindf = pd.read_csv(DIR)
traindf["id"] = traindf['GalaxyID'].astype(str).apply(append_ext)

def random_input(img):
    shape = img.shape[:2]
    left = int(shape[0]/4)
    top = int(shape[1]/4)
    img = img[left:left*3,top:top*3,:]
    image = cv2.resize(img, shape, interpolation = cv2.INTER_CUBIC)
    image = img_to_array(image)
    return image
  
datagen = ImageDataGenerator(
    #featurewise_center=True,
    #featurewise_std_normalization=True,
    rescale=1. / 255,
    rotation_range=90,
    width_shift_range=0.1,
    height_shift_range=0.1,
    brightness_range = (0.9, 1.2),
    horizontal_flip=True,
    vertical_flip=True,
    validation_split = 0.15,
    preprocessing_function = random_input,
)

valid_datagen=ImageDataGenerator(
    rescale=1./255,
    validation_split=0.15, 
    preprocessing_function = random_input)

train_generator = datagen.flow_from_dataframe(
    dataframe=traindf,
    directory=train_path,
    x_col="id",
    y_col=classes,
    subset="training",
    batch_size=32,
    seed=123,
    shuffle=False,
    class_mode="raw",
    target_size=(224,224))


STEP_SIZE_TRAIN = train_generator.n // train_generator.batch_size

valid_generator = valid_datagen.flow_from_dataframe(
    dataframe=traindf,
    directory=train_path,
    x_col="id",
    y_col=classes,
    subset="validation",
    batch_size=32,
    seed=123,
    shuffle=False,
    class_mode="raw",
    target_size=(224, 224))

STEP_SIZE_VALID = valid_generator.n // valid_generator.batch_size

In [ ]:
mean = np.array([123.68, 116.779, 103.939], dtype="float32")
train_generator.mean=train_generator
valid_generator.mean=valid_generator

In [ ]:
from keras.models import Sequential, Model,load_model
from keras.applications.resnet_v2 import ResNet50V2
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.layers import Add,Input,Dense,Dropout,BatchNormalization,Activation,Flatten,Conv2D,MaxPooling2D,ZeroPadding2D,Lambda,AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.optimizers import SGD, RMSprop, Adam, Adagrad
from keras.initializers import glorot_uniform

In [ ]:
!pip install efficientnet

In [ ]:
import efficientnet.keras as efn 
import keras.backend as backend

def rmse(y_true, y_pred):
        return backend.sqrt(backend.mean(backend.square(y_pred - y_true))) 



In [ ]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy()

In [ ]:
with strategy.scope():
  def build_model():
      eff1 = efn.EfficientNetB1(weights='imagenet',include_top=False,input_shape=(256,256,3))
      model = Sequential()
      model.add(eff1)
      model.add(GlobalAveragePooling2D())
      model.add(Dropout(0.25))
      model.add(Dense(64, activation='relu'))
      model.add(Dense(37, activation='sigmoid'))
      opt = Adam(lr=1.5e-4)
      model.compile(loss=rmse,
                  optimizer=opt, 
                  metrics=[rmse, 'accuracy'])
      return model
model = build_model()

In [ ]:
model.summary()

In [ ]:
from keras.callbacks import Callback
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping, ReduceLROnPlateau, CSVLogger


class LossHistory(Callback):
    def on_train_begin(self, logs={}):
        self.losses = []
        self.val_losses = []
        self.rmse = []

    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))
        self.rmse.append(logs.get('rmse'))


early_stopping = EarlyStopping(
    monitor='val_loss', patience=10, verbose=1, mode='auto')

history = LossHistory()

from keras.callbacks import ModelCheckpoint
checkpointer = ModelCheckpoint(
    filepath='/content/drive/My Drive/results/weights_efficientnetB1_TPU_FC64.hdf5', verbose=2, save_best_only=True)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=4,)

csv_logger = CSVLogger('/content/drive/My Drive/results/history/training_efficientnetB1_TPU_FC64.csv')

In [ ]:
hist = model.fit_generator(
    train_generator,
    steps_per_epoch=STEP_SIZE_TRAIN,
    validation_data=valid_generator,
    validation_steps=STEP_SIZE_VALID,
    epochs=50,
    callbacks=[history, checkpointer, reduce_lr, early_stopping, csv_logger])
        